In [259]:
import pandas as pd
from pandas_profiling import ProfileReport
import ipywidgets as widgets
from ipywidgets import interact
import plotly.express as px
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import svm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [228]:
def read(path):
    """
    wczytanie danych i usunięcie zbędnych kolumn
    """
    df = pd.read_csv(path)
    df = df.drop(['email', 'phone-number', 'credit_card', 'name'], axis = 1)
    # zmiana na 0-1 zmienną
    df.loc[(df.hotel == 'City Hotel'), 'hotel'] = 0
    df.loc[(df.hotel == 'Resort Hotel'), 'hotel'] = 1
    return df

In [250]:
df = read(r'C:\Users\User\Desktop\repozytoria\wm_2022\projects\magdas\data\hotel_booking.csv')

In [78]:
# profile = df.drop(['hotel', 'arrival_date_month', 'meal', 'country', 'market_segment', 'distribution_channel', 'reserved_room_type',
#                  'assigned_room_type', 'deposit_type', 'customer_type', 'reservation_status', 'reservation_status_date',
#                  'name'], axis = 1)

In [255]:
#prof = ProfileReport(df,explorative=True)
#prof.to_file("output.pdf")

In [124]:
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,name,email,phone-number,credit_card
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,Transient,0.0,0,0,Check-Out,2015-07-01,Ernest Barnes,Ernest.Barnes31@outlook.com,669-792-1661,************4322
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,Transient,0.0,0,0,Check-Out,2015-07-01,Andrea Baker,Andrea_Baker94@aol.com,858-637-6955,************9157
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,Transient,75.0,0,0,Check-Out,2015-07-02,Rebecca Parker,Rebecca_Parker@comcast.net,652-885-2745,************3734
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,Transient,75.0,0,0,Check-Out,2015-07-02,Laura Murray,Laura_M@gmail.com,364-656-8427,************5677
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,Transient,98.0,0,1,Check-Out,2015-07-03,Linda Hines,LHines@verizon.com,713-226-5883,************5498


In [130]:
@interact(kolumna = ['lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 
       'previous_cancellations',
       'previous_bookings_not_canceled', 'booking_changes', 
       'days_in_waiting_list', 'adr','required_car_parking_spaces', 'total_of_special_requests'])
def draw_box_plot(kolumna):
    histogram = px.histogram(data_frame=df, x=kolumna, barmode='relative', barnorm=None, histnorm='percent'
             , color_discrete_sequence = ['lightblue']
             , title="Histogram zmiennej " + str(kolumna)
             )
    boxplot = px.box(data_frame=df,y=kolumna,title="Boxplot zmiennej " + str(kolumna) )
    return histogram.show(), boxplot.show()

interactive(children=(Dropdown(description='kolumna', options=('lead_time', 'arrival_date_year', 'arrival_date…

In [205]:
def IQR(dataframe, column):
    q3, q1 = np.nanpercentile(dataframe[column], [75 ,25])
    iqr = q3 - q1
    dataframe.loc[((dataframe[column] > (q3 + iqr)) | (dataframe[column] < (q3 - iqr))), column] = None
    return dataframe

In [213]:
def outliery(dataframe):
    dataframe = IQR(dataframe, 'stays_in_weekend_nights')
    dataframe = IQR(dataframe, 'stays_in_week_nights')
    dataframe.loc[(dataframe['adults'] > 10), 'adults'] = None
    dataframe.loc[(dataframe['children'] > 4), 'children'] = None
    dataframe.loc[(dataframe['babies'] > 3), 'babies'] = None
    dataframe.loc[(dataframe['adr'] > 600), 'adr'] = None
    dataframe = IQR(dataframe, 'adr')
    dataframe.loc[(dataframe['required_car_parking_spaces'] > 3), 'required_car_parking_spaces'] = None
    return dataframe

In [251]:
df = outliery(df)

In [244]:
df.isnull().sum()

hotel                                  0
is_canceled                            0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights              344
stays_in_week_nights               12498
adults                                12
children                               5
babies                                 2
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

In [218]:
np.mean(df.adr)

117.11374691822633

In [252]:
def braki(dataframe):
    # usuwamy zmienną company, bo mamy 94% braków danych
    dataframe = dataframe.drop(['company'], axis = 1)
    dataframe.loc[(dataframe.agent.isnull()), 'agent'] = 'Brak'
    dataframe.loc[(dataframe.adr.isnull()), 'adr'] = np.mean(dataframe.adr)
    dataframe.loc[(dataframe.stays_in_week_nights.isnull()), 'stays_in_week_nights'] = np.mean(dataframe.stays_in_week_nights)
    dataframe.loc[(dataframe.lead_time.isnull()), 'stays_in_week_nights'] = np.mean(dataframe.lead_time)
    # Usuwamy pozostałe rekordy < 1%
    dataframe = dataframe.dropna()
    return dataframe

In [253]:
df = braki(df)

In [254]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118540 entries, 0 to 119389
Data columns (total 31 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           118540 non-null  object 
 1   is_canceled                     118540 non-null  int64  
 2   lead_time                       118540 non-null  int64  
 3   arrival_date_year               118540 non-null  int64  
 4   arrival_date_month              118540 non-null  object 
 5   arrival_date_week_number        118540 non-null  int64  
 6   arrival_date_day_of_month       118540 non-null  int64  
 7   stays_in_weekend_nights         118540 non-null  float64
 8   stays_in_week_nights            118540 non-null  float64
 9   adults                          118540 non-null  float64
 10  children                        118540 non-null  float64
 11  babies                          118540 non-null  float64
 12  meal            

In [269]:
y = df.is_canceled

In [270]:
X = df.drop(['is_canceled'], axis = 1)

In [271]:
# podział na treningowy i pozostałe
x_train, x_Combine, y_train, y_Combine = train_test_split(X,y,
                                              train_size=0.7,
                                              random_state=1234)
x_val, x_test, y_val, y_test = train_test_split(x_Combine,y_Combine,
                                              train_size=0.8,
                                              random_state=42)

In [280]:
num = []
obj = []
for column in x_train.columns:
    if x_train[column].dtypes != 'object':
        num.append(column)
    else:
        obj.append(column)

In [283]:
x_train_n = x_train.drop(obj, axis = 1)
x_val_n = x_val.drop(obj, axis = 1)

In [284]:
x_train_o = x_train.drop(num, axis = 1)
x_val_o = x_val.drop(num, axis = 1)

In [287]:
# Standaryzacja
x_train_n = x_train.drop(obj, axis = 1)
scaler = preprocessing.StandardScaler()
scaler.fit(x_train_n)
x_traint = scaler.transform(x_train_n)
x_valt = scaler.transform(x_val_n)

In [289]:
#X_train = pd.concat([x_train_n, x_train_o], axis=1)
#X_val = pd.concat([x_val_n, x_val_o], axis=1)

In [292]:
clf = svm.SVC(kernel='linear', C=1)
clf.fit(x_train_n, y_train)

SVC(C=1, kernel='linear')

In [295]:
print(clf.score(x_train_n, y_train))
print(clf.score(x_val_n , y_val))

0.6943406686109571
0.6929593307321874
